In [1]:
# 라이브러리 import, 데이터 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/submission.csv')
model = DecisionTreeRegressor()

In [2]:
# 데이터 확인하기
train.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [3]:
# 데이터 타입, 개수, 결측치 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


<h1>결측치가 존재하는 테이블</h1>
<li>temperature</li>
<li>precipitation</li>
<li>windspeed</li>
<li>humidity</li>
<li>visibility</li>
<li>ozone</li>
<li>pm10</li>
<li>pm2.5</li>

In [4]:
# 결측치를 이전 데이터와 이후 데이터의 평균으로 보간
train = train.interpolate(method='linear')
test = test.interpolate(method='linear')

In [5]:
# features 와 target 지정
columns = [
            'hour',
            'hour_bef_temperature',
            'hour_bef_precipitation',
            'hour_bef_humidity',
            'hour_bef_visibility',
            'hour_bef_ozone',
            'hour_bef_pm10',
            'hour_bef_pm2.5'
           ]

X_train = train[columns]
Y_train = train['count']
test = test[columns]

In [6]:
# Dicision Tree 방식
model = DecisionTreeRegressor()
model.fit(X_train,Y_train)
accuracies = cross_val_score(model,X_train,Y_train,cv=5)
print(accuracies)
print("accuracy(mean)=%0.3f, 표준편차=%0.3f"%(accuracies.mean()*100,accuracies.std()))
pred = model.predict(test)
pred[:5]

[0.57540682 0.56533566 0.60287348 0.61687596 0.63575595]
accuracy(mean)=59.925, 표준편차=0.026


array([127., 142.,   1.,  18.,   2.])

In [7]:
# 랜덤 포레스트 방식
model = RandomForestRegressor()
model.fit(X_train,Y_train)
accuracies = cross_val_score(model,X_train,Y_train,cv=5)
print(accuracies)
print("accuracy(mean)=%0.3f, 표준편차=%0.3f"%(accuracies.mean()*100,accuracies.std()))
pred = model.predict(test)
pred[:5]

[0.79329527 0.77102076 0.81763452 0.76822311 0.77582432]
accuracy(mean)=78.520, 표준편차=0.018


array([ 99.05, 215.93,  84.79,  30.67,  74.74])

In [8]:
#제출 파일 생성
submission['count'] = pred
submission.to_csv('sub.csv',index=False)